In [2]:
# Python 3
import http.client, urllib.parse
import json
import pandas as pd
from tqdm import tqdm
tqdm.pandas(desc='geo')

def geocode(_add):
    conn = http.client.HTTPConnection('api.positionstack.com')

    params = urllib.parse.urlencode({
        'access_key': '68c727007d3bcce626e3bedfb56a092f',
        'query': _add,
        'country': 'US',
        'limit': 1,
        })

    conn.request('GET', '/v1/forward?{}'.format(params))

    res = conn.getresponse()
    try:
        data = res.read()
        return json.loads(data)['data'][0]['region']
    except :
        return ''


In [3]:
df = pd.read_csv('/mnt/data/shared/notfound_left.csv',index_col=0)

In [4]:
df=df[df.state.isnull()]

In [5]:
df.drop_duplicates(subset=['place_name'],inplace=True)

In [6]:
df.reset_index(drop=True,inplace=True)

In [7]:
dflst=[df[i:i+20] for i in range(0,df.shape[0],20)]

for each_df in tqdm(dflst):

    each_df['state'] = each_df['place_name'].progress_apply(lambda x: geocode(x))
    each_df.to_csv('nofound_left_positionstack.csv', mode='a')

geo: 100%|██████████| 20/20 [00:13<00:00,  1.51it/s]
/tmp/ipykernel_973051/4125079736.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  each_df['state'] = each_df['place_name'].progress_apply(lambda x: geocode(x))
100%|██████████| 229/229 [48:02<00:00, 12.59s/it]
